In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

MessageError: ignored

In [ ]:
%cd gdrive/MyDrive/Colab Notebooks/Airbnb

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'
PALETTE = px.colors.qualitative.Prism
MAIN_COLOR = 'rgb(231,41,138)'

In [ ]:
def create_df(file_name, cols, na_cols=None):
  df = pd.read_csv(file_name).drop(cols, axis=1)
  df[na_cols] = df[na_cols].fillna('No name')
  return df

In [ ]:
cols = ['license', 'neighbourhood_group']
na_cols = ['name']
df_listing = create_df('listings_1.csv', cols, na_cols)

In [ ]:
df_listing.isna().sum().sort_values(ascending=False)

In [ ]:
df_reviews = pd.read_csv('reviews_2023_03.csv', parse_dates=['date'])
df_final = df_reviews.merge(df_listing, left_on='listing_id', right_on='id', how='inner').drop(['id_x', 'id_y'], axis=1)

In [ ]:
df_final.isna().sum()

In [ ]:
df_final[df_final['comments'].isna() == True]

In [ ]:
df_final.dropna(inplace=True)

In [ ]:
df_final.isna().sum()

In [ ]:
df_final['reviewer_id'].value_counts()

In [ ]:
df_final.query('reviewer_id == 348992094').head(3)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_final['listing_id_encod'] = le.fit_transform(df_final['listing_id'])

In [ ]:
px.histogram(df_final, x='listing_id_encod')

In [ ]:
num_reviews = (df_final.groupby(['listing_id_encod', 'neighbourhood'])
          .size().sort_values(ascending=False)
          .reset_index()
          .rename(columns={0: 'num_reviews'}))
num_reviews

In [ ]:
px.bar(num_reviews.nlargest(50, 'num_reviews'), y='num_reviews', 
       color='neighbourhood', color_discrete_sequence=PALETTE)

In [ ]:
fig = px.bar(num_reviews.groupby('neighbourhood')['num_reviews'].sum().sort_values(ascending=True), orientation='h')
fig.update_traces(marker_color=MAIN_COLOR)

In [ ]:
df_final[df_final['host_name'].str.contains("UNITY") == True].sort_values('date', ascending=False)